In [1]:
#need to pip install google-cloud-batch first time you run
import pandas as pd
import os
import shutil
import numpy as np
import subprocess
import glob
import sys
import gzip
from google.cloud import batch_v1

# Supress copy warning.
pd.options.mode.chained_assignment = None

In [2]:
! gcsfuse --dir-mode 777 --file-mode 777 --implicit-dirs gp2_genotools_data ~/gp2_genotools_data

{"time":"07/03/2025 02:04:03.442142","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /home/levineks/gp2_genotools_data\n"}


In [3]:
study = 'VETPD'

In [4]:
key = pd.read_csv(f'/home/levineks/gp2_genotools_data/clinical/GP2_master_key_MARCH_2025.txt', sep = '\t', low_memory = False)
key = key[key['study'] == study]
print(len(key))
key.head()

382


,GP2sampleID_old,GP2sampleID,SentrixBarcode_A,SentrixPosition_A,study_old,study,sample_id,clinical_id,study_arm,study_type,...,family_history_other,family_history_pd,family_index,family_index_relationship,filename,FID,IID,pheno,sex_for_qc,GDPR
91402,NaN,VETPD_000001_s1,209042130073,R02C01,NaN,VETPD,GP2Z0001.1,GP2Z0001,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130073_R02C01,2,1,0
91403,NaN,VETPD_000002_s1,209042130008,R06C01,NaN,VETPD,GP2Z0002.1,GP2Z0002,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130008_R06C01,2,1,0
91404,NaN,VETPD_000003_s1,209042130073,R07C01,NaN,VETPD,GP2Z0003.1,GP2Z0003,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130073_R07C01,2,1,0
91405,NaN,VETPD_000004_s1,209042130062,R06C01,NaN,VETPD,GP2Z0004.1,GP2Z0004,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130062_R06C01,2,1,0
91406,NaN,VETPD_000005_s1,209042130055,R04C01,NaN,VETPD,GP2Z0005.2,GP2Z0005,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130055_R04C01,2,1,0


In [5]:
# If previous samples exist
test = pd.read_csv(f'/home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_{study}.fam', sep = '\s+', header = None)
print(study)
print(test.shape)
print(test.head())
list_included = list(test[1])
print(len(list_included))

VETPD
(190, 6)
   0                1  2  3  4  5
0  0  VETPD_000028_s1  0  0  1 -9
1  0  VETPD_000029_s1  0  0  1 -9
2  0  VETPD_000030_s1  0  0  1 -9
3  0  VETPD_000031_s1  0  0  1 -9
4  0  VETPD_000032_s1  0  0  1 -9
190


In [6]:
#The samples we still need to process are the ones that were NOT included in the prevous merged file
key = key[~key['GP2sampleID'].isin(list_included)]
key

,GP2sampleID_old,GP2sampleID,SentrixBarcode_A,SentrixPosition_A,study_old,study,sample_id,clinical_id,study_arm,study_type,...,family_history_other,family_history_pd,family_index,family_index_relationship,filename,FID,IID,pheno,sex_for_qc,GDPR
91592,NaN,VETPD_000193_s1,209281700115,R01C01,NaN,VETPD,GP2Z0061.1,GP2Z0061,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R01C01,2,1,0
91593,NaN,VETPD_000194_s1,209281700115,R02C01,NaN,VETPD,GP2Z0062.1,GP2Z0062,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R02C01,2,1,0
91594,NaN,VETPD_000195_s1,209281700115,R03C01,NaN,VETPD,GP2Z0063.1,GP2Z0063,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R03C01,2,1,0
91595,NaN,VETPD_000196_s1,209281700115,R04C01,NaN,VETPD,GP2Z0064.1,GP2Z0064,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R04C01,2,1,0
91596,NaN,VETPD_000197_s1,209281700007,R06C01,NaN,VETPD,GP2Z0068.1,GP2Z0068,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700007_R06C01,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91779,NaN,VETPD_000380_s1,209281710051,R08C01,NaN,VETPD,GP2Z0731.1,GP2Z0731,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R08C01,2,1,0
91780,NaN,VETPD_000381_s1,209281710051,R07C01,NaN,VETPD,GP2Z0732.1,GP2Z0732,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R07C01,2,1,0
91781,NaN,VETPD_000382_s1,209281710051,R06C01,NaN,VETPD,GP2Z0733.1,GP2Z0733,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R06C01,2,1,0
91782,NaN,VETPD_000383_s1,209281710051,R05C01,NaN,VETPD,GP2Z0734.1,GP2Z0734,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R05C01,2,1,0


In [7]:
print(len(key))
barcode_list = list(set(list(key['SentrixBarcode_A'])))
print(len(barcode_list))
print(barcode_list)

192
24
['209281700022', '209281700020', '209281710133', '209281700092', '209281700007', '209281700115', '209281700035', '209281700163', '209281700041', '209281710005', '209281700017', '209281710051', '209281700145', '209281700123', '209281700112', '209281700106', '209281710050', '209281700107', '209281700069', '209281710135', '209281700119', '209281700122', '209281700040', '209281700084']


In [8]:
# Create shell script to execute -- convert IDATs to ped
count = 0 # for Job ID count

# Establish paths for inputs/outputs

iaap = f'./gp2_genotools_data/ilmn_files/iaap-cli/iaap-cli'
bpm = f'./gp2_genotools_data/ilmn_files/NeuroBooster_20042459_A2.bpm'
egt = f'./gp2_genotools_data/ilmn_files/recluster_09272022.egt'
raw_plink_path = f'./gp2_genotools_data/ped_bed'
idat_path = f'./gp2_idats/${{BARCODE}}'

idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{raw_plink_path}/ \
-f {idat_path} \
-p \
-t 8'

with open(f'./gp2_genotools_data/batch_files/convert_idats_to_ped.sh', 'w') as f:
    f.write('#!/bin/bash\n\n')
    f.write('BARCODE=$1\n\n')
    f.write(f'{idat_to_ped_cmd}\n')
    f.close()

print(f"Preparing bash script to convert IDATs to ped for {study}.")

Preparing bash script to convert IDATs to ped for VETPD.


In [9]:
# 2 buckets mounted: 1 for inputs, 1 for outputs
def create_script_job_with_buckets(
    project_id: str, region: str, job_name: str, bucket_name_input: str, bucket_name_output: str, script_text: str) -> batch_v1.Job:
    """
    This method shows how to create a sample Batch Job that will run
    a simple command on Cloud Compute instances.

    Args:
        project_id: project ID or project number of the Cloud project you want to use.
        region: name of the region you want to use to run the job. Regions that are
            available for Batch are listed on: https://cloud.google.com/batch/docs/get-started#locations
        job_name: the name of the job that will be created.
            It needs to be unique for each project and region pair.
        bucket_name: name of the bucket to be mounted for your Job.

    Returns:
        A job object representing the job created.
    """
    client = batch_v1.BatchServiceClient()

    # Define what will be done as part of the job.
    task = batch_v1.TaskSpec()
    runnable = batch_v1.Runnable()

    # This would replace runnable script
    # runnable.container = batch_v1.Runnable.Container()
    # runnable.container.image_uri = docker_image

    runnable.script = batch_v1.Runnable.Script()
    runnable.script.text = script_text
    task.runnables = [runnable]

    # Define the first GCS bucket 
    gcs_bucket_input = batch_v1.GCS(remote_path=bucket_name_input)
    gcs_volume_input = batch_v1.Volume(gcs=gcs_bucket_input, mount_path="./gp2_idats")

    # Define the second GCS bucket 
    gcs_bucket_output = batch_v1.GCS(remote_path=bucket_name_output)
    gcs_volume_output = batch_v1.Volume(gcs=gcs_bucket_output, mount_path="./gp2_genotools_data")

    # Add both volumes to the task
    task.volumes = [gcs_volume_input, gcs_volume_output]

    # We can specify what resources are requested by each task.
    resources = batch_v1.ComputeResource()
    resources.cpu_milli = 8000  #in milliseconds per cpu-second. This means the task requires 50% of a single CPUs; 2000 = 2 CPUs
    resources.memory_mib = 32768 # 4 GiB of memory (4096 MiB - mebibyte); 32768 MiB = 32 GB; 16384 = 16 GB
    task.compute_resource = resources

    task.max_retry_count = 0 # I'd suggest changing this to 0; otherwise, it tries two times before it fails
    task.max_run_duration = "3600s" # 24 hours = 86400s; 1 hour = 3600s; 8 hours = 28800s

    # Tasks are grouped inside a job using TaskGroups.
    # Currently, it's possible to have only one task group.
    group = batch_v1.TaskGroup()
    group.task_count = 1
    group.task_spec = task

    # Policies are used to define on what kind of virtual machines the tasks will run on.
    # Read more about machine types here: https://cloud.google.com/compute/docs/machine-types
    allocation_policy = batch_v1.AllocationPolicy()
    policy = batch_v1.AllocationPolicy.InstancePolicy()
    policy.machine_type = "e2-standard-8"
    instances = batch_v1.AllocationPolicy.InstancePolicyOrTemplate()
    instances.policy = policy
    allocation_policy.instances = [instances]

    job = batch_v1.Job()
    job.task_groups = [group]
    job.allocation_policy = allocation_policy
    job.labels = {"env": "testing", "type": "script", "mount": "bucket"}
    # We use Cloud Logging as it's an out of the box available option
    job.logs_policy = batch_v1.LogsPolicy()
    job.logs_policy.destination = batch_v1.LogsPolicy.Destination.CLOUD_LOGGING

    create_request = batch_v1.CreateJobRequest()
    create_request.job = job
    create_request.job_id = job_name
    # The job's parent is the region in which the job will run
    create_request.parent = f"projects/{project_id}/locations/{region}"

    return client.create_job(create_request)

In [10]:
# Double check the shell script to make sure it does what you need and paths are correct
! cat ./gp2_genotools_data/batch_files/convert_idats_to_ped.sh

#!/bin/bash

BARCODE=$1

./gp2_genotools_data/ilmn_files/iaap-cli/iaap-cli gencall ./gp2_genotools_data/ilmn_files/NeuroBooster_20042459_A2.bpm ./gp2_genotools_data/ilmn_files/recluster_09272022.egt ./gp2_genotools_data/ped_bed/ -f ./gp2_idats/${BARCODE} -p -t 8


In [11]:
barcode_list = list(set(list(key['SentrixBarcode_A'])))
print(len(barcode_list))
print(barcode_list)

24
['209281700022', '209281700020', '209281710133', '209281700092', '209281700007', '209281700115', '209281700035', '209281700163', '209281700041', '209281710005', '209281700017', '209281710051', '209281700145', '209281700123', '209281700112', '209281700106', '209281710050', '209281700107', '209281700069', '209281710135', '209281700119', '209281700122', '209281700040', '209281700084']


In [12]:
study

'VETPD'

In [13]:
# Update the count with the last job # that you ran:
count = 10

# Define the number of codes to process per job (8)
codes_per_job = 5

# Split the filename_list into chunks
from itertools import zip_longest

# Helper function to chunk the list into groups of 8
def chunk_list(iterable, n):
    args = [iter(iterable)] * n
    return zip_longest(*args)

# Loop through each chunk of 8 codes and create a single job
for chunk in chunk_list(barcode_list, codes_per_job):
    # Filter out any None values from the last incomplete chunk
    codes = [code for code in chunk if code is not None]
    
    # Generate a unique job name
    count += 1
    job_name = f'idattoped{study.lower()}{count}'
    
    # Create a combined script for the N codes
    script = f"""
        #!/bin/bash

        # Install and activate PLINK
        chmod +x ./gp2_genotools_data/batch_files/plink2_module.sh
        ./gp2_genotools_data/batch_files/plink2_module.sh
    """

    # Add commands for each code in the chunk
    for code in codes:
        script += f"""
        # Make analysis script executable and run for a specific code
        chmod +x ./gp2_genotools_data/batch_files/convert_idats_to_ped.sh
        ./gp2_genotools_data/batch_files/convert_idats_to_ped.sh {code}
        """

    # Create the job with the combined script
    create_script_job_with_buckets(
        project_id = "gp2-release-terra", 
        region = 'europe-west4', 
        job_name = job_name, 
        bucket_name_input= 'gp2_idats', bucket_name_output= 'gp2_genotools_data', 
        script_text = script
    )

In [21]:
# Initialize the client
#count = 1
client = batch_v1.BatchServiceClient()
job_id = f'idattoped{study.lower()}{count}'

# Define the job name (replace with your project ID, location, and job ID)
project_id = "gp2-release-terra"
region = 'europe-west4'
job_name = f"projects/{project_id}/locations/{region}/jobs/{job_id}"

# Get the job details
job = client.get_job(name=job_name)

# Check the status
job_status = job.status.state
print(study)
print("Last job:", count)
print(f"Job Status: {batch_v1.JobStatus.State(job_status).name}")

VETPD
Last job: 15
Job Status: SUCCEEDED


In [22]:
# can also check on job here: https://console.cloud.google.com/batch/jobs?project=gp2-release-terra&inv=1&invt=Abj2Qw

## check if any ped job has failed

In [22]:
study_list = [study]
print(study)
print(len(key))

VETPD
192


In [23]:
# copy map file to match name of each ped
missing_peds = []
map_file = f'./gp2_genotools_data/ped_bed/NeuroBooster_20042459_A2.map'
raw_plink_path = f'./gp2_genotools_data/ped_bed'
missing_idat_dir = f'./gp2_genotools_data/missing'

for cohort in study_list:
    df = key[key['study']==cohort]
    
    missing_cnt = 0

    for filename in df.IID:
        ped = f'{raw_plink_path}/{filename}.ped'
        out_map = f'{raw_plink_path}/{filename}.map'
        if os.path.isfile(ped):
            shutil.copyfile(src=map_file, dst=out_map)
        else:
            missing_cnt += 1
            missing_peds.append(filename)
            
    print(f"There is {missing_cnt} missing ped file for {cohort}.")

with open(f'{missing_idat_dir}/missing_peds_{cohort}.txt', 'w') as f:
    for m_ped in missing_peds:
        f.write(f'{m_ped}\n')
f.close()

There is 0 missing ped file for VETPD.


In [24]:
missing = key[key['IID'].isin(missing_peds)]
missing.SentrixBarcode_A.value_counts()
#missing.to_csv('/home/levineks/gp2_genotools_data/missing/SP-BCN2_n8_missing.csv', header = True, index = False)

Series([], Name: SentrixBarcode_A, dtype: int64)

In [25]:
key = key[~key['IID'].isin(missing_peds)]
key

,GP2sampleID_old,GP2sampleID,SentrixBarcode_A,SentrixPosition_A,study_old,study,sample_id,clinical_id,study_arm,study_type,...,family_history_other,family_history_pd,family_index,family_index_relationship,filename,FID,IID,pheno,sex_for_qc,GDPR
91592,NaN,VETPD_000193_s1,209281700115,R01C01,NaN,VETPD,GP2Z0061.1,GP2Z0061,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R01C01,2,1,0
91593,NaN,VETPD_000194_s1,209281700115,R02C01,NaN,VETPD,GP2Z0062.1,GP2Z0062,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R02C01,2,1,0
91594,NaN,VETPD_000195_s1,209281700115,R03C01,NaN,VETPD,GP2Z0063.1,GP2Z0063,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R03C01,2,1,0
91595,NaN,VETPD_000196_s1,209281700115,R04C01,NaN,VETPD,GP2Z0064.1,GP2Z0064,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R04C01,2,1,0
91596,NaN,VETPD_000197_s1,209281700007,R06C01,NaN,VETPD,GP2Z0068.1,GP2Z0068,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700007_R06C01,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91779,NaN,VETPD_000380_s1,209281710051,R08C01,NaN,VETPD,GP2Z0731.1,GP2Z0731,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R08C01,2,1,0
91780,NaN,VETPD_000381_s1,209281710051,R07C01,NaN,VETPD,GP2Z0732.1,GP2Z0732,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R07C01,2,1,0
91781,NaN,VETPD_000382_s1,209281710051,R06C01,NaN,VETPD,GP2Z0733.1,GP2Z0733,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R06C01,2,1,0
91782,NaN,VETPD_000383_s1,209281710051,R05C01,NaN,VETPD,GP2Z0734.1,GP2Z0734,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R05C01,2,1,0


In [26]:
key.study.value_counts()

VETPD    192
Name: study, dtype: int64

# Create bed files

In [27]:
# 1 bucket mounted for inputs/ouput

def create_script_job_with_bucket(
    project_id: str, region: str, job_name: str, bucket_name: str, script_text: str) -> batch_v1.Job:
    """
    This method shows how to create a sample Batch Job that will run
    a simple command on Cloud Compute instances.

    Args:
        project_id: project ID or project number of the Cloud project you want to use.
        region: name of the region you want to use to run the job. Regions that are
            available for Batch are listed on: https://cloud.google.com/batch/docs/get-started#locations
        job_name: the name of the job that will be created.
            It needs to be unique for each project and region pair.
        bucket_name: name of the bucket to be mounted for your Job.

    Returns:
        A job object representing the job created.
    """
    client = batch_v1.BatchServiceClient()

    # Define what will be done as part of the job.
    task = batch_v1.TaskSpec()
    runnable = batch_v1.Runnable()
    runnable.script = batch_v1.Runnable.Script()
    runnable.script.text = script_text
    task.runnables = [runnable]

    gcs_bucket = batch_v1.GCS()
    gcs_bucket.remote_path = bucket_name
    gcs_volume = batch_v1.Volume()
    gcs_volume.gcs = gcs_bucket
    gcs_volume.mount_path = "./gp2_genotools_data"
    task.volumes = [gcs_volume]

    # We can specify what resources are requested by each task.
    resources = batch_v1.ComputeResource()
    resources.cpu_milli = 8000  #8000 milliseconds = 8 CPUs
    resources.memory_mib = 32768 # 200192 MiB = 200 GB of memory; 32768 MiB = 32 GB; 16384 = 16 GB
    task.compute_resource = resources

    task.max_retry_count = 0
    #task.max_run_duration = "3600s" # 24 hours = 86400s; 1 hour = 3600s; 8 hours = 28800s
    task.max_run_duration = "28800s" # 24 hours = 86400s; 1 hour = 3600s; 8 hours = 28800s

    # Tasks are grouped inside a job using TaskGroups.
    # Currently, it's possible to have only one task group.
    group = batch_v1.TaskGroup()
    group.task_count = 1
    group.task_spec = task

    # Policies are used to define on what kind of virtual machines the tasks will run on.
    # In this case, we tell the system to use "e2-standard-4" machine type.
    # Read more about machine types here: https://cloud.google.com/compute/docs/machine-types
    allocation_policy = batch_v1.AllocationPolicy()
    policy = batch_v1.AllocationPolicy.InstancePolicy()
    policy.machine_type = "e2-standard-8"
    instances = batch_v1.AllocationPolicy.InstancePolicyOrTemplate()
    instances.policy = policy
    allocation_policy.instances = [instances]

    job = batch_v1.Job()
    job.task_groups = [group]
    job.allocation_policy = allocation_policy
    job.labels = {"env": "testing", "type": "script", "mount": "bucket"}
    # We use Cloud Logging as it's an out of the box available option
    job.logs_policy = batch_v1.LogsPolicy()
    job.logs_policy.destination = batch_v1.LogsPolicy.Destination.CLOUD_LOGGING

    create_request = batch_v1.CreateJobRequest()
    create_request.job = job
    create_request.job_id = job_name
    # The job's parent is the region in which the job will run
    create_request.parent = f"projects/{project_id}/locations/{region}"

    return client.create_job(create_request)

In [28]:
# Create shell script to execute -- convert ped to bed
count = 0 # for Job ID count

# Establish paths for inputs/outputs

ped_file = f'./gp2_genotools_data/ped_bed/${{FILENAME}}'

make_bed_cmd = f"/home/levineks/bin/plink1.9/plink --file {ped_file} --make-bed --out {ped_file}"

with open(f'./gp2_genotools_data/batch_files/convert_ped_to_bed.sh', 'w') as f:
    f.write('#!/bin/bash\n\n')
    f.write('FILENAME=$1\n\n')
    f.write(f'{make_bed_cmd}\n')
    f.close()

print(f"Preparing bash script to convert ped to bed for {study}.")

Preparing bash script to convert ped to bed for VETPD.


In [29]:
# Double check the shell script to make sure it does what you need and paths are correct
! cat ./gp2_genotools_data/batch_files/convert_ped_to_bed.sh

#!/bin/bash

FILENAME=$1

/home/levineks/bin/plink1.9/plink --file ./gp2_genotools_data/ped_bed/${FILENAME} --make-bed --out ./gp2_genotools_data/ped_bed/${FILENAME}


In [30]:
filename_list = list(key['IID'])
print(len(filename_list))
print(filename_list)

192
['209281700115_R01C01', '209281700115_R02C01', '209281700115_R03C01', '209281700115_R04C01', '209281700007_R06C01', '209281700115_R05C01', '209281700007_R07C01', '209281700007_R08C01', '209281700040_R01C01', '209281700115_R06C01', '209281700115_R07C01', '209281700115_R08C01', '209281700092_R01C01', '209281700092_R02C01', '209281700092_R03C01', '209281700041_R08C01', '209281710051_R01C01', '209281700092_R04C01', '209281700092_R05C01', '209281700040_R04C01', '209281700092_R06C01', '209281700040_R05C01', '209281700092_R07C01', '209281700122_R02C01', '209281700092_R08C01', '209281700122_R01C01', '209281700040_R06C01', '209281700040_R07C01', '209281700035_R04C01', '209281700035_R05C01', '209281700122_R03C01', '209281700007_R05C01', '209281700040_R08C01', '209281700122_R04C01', '209281700084_R01C01', '209281710133_R01C01', '209281710133_R02C01', '209281700084_R02C01', '209281700040_R02C01', '209281700122_R05C01', '209281700122_R06C01', '209281700084_R03C01', '209281710133_R03C01', '20928

In [31]:
# Update the count with the last job # that you ran:
count = 10

# Define the number of codes to process per job (8)(Up this amount x3)
codes_per_job = 60

# Split the filename_list into chunks
from itertools import zip_longest

# Helper function to chunk the list into groups of 8
def chunk_list(iterable, n):
    args = [iter(iterable)] * n
    return zip_longest(*args)

# Loop through each chunk of 8 codes and create a single job
for chunk in chunk_list(filename_list, codes_per_job):
    # Filter out any None values from the last incomplete chunk
    codes = [code for code in chunk if code is not None]
    
    # Generate a unique job name
    count += 1
    job_name = f'pedtobed{study.lower()}{count}'
    
    # Create a combined script for the N codes
    script = f"""
        #!/bin/bash

        # Install and activate PLINK
        chmod +x ./gp2_genotools_data/batch_files/plink_module.sh
        ./gp2_genotools_data/batch_files/plink_module.sh
    """

    # Add commands for each code in the chunk
    for code in codes:
        script += f"""
        # Make analysis script executable and run for a specific code
        chmod +x ./gp2_genotools_data/batch_files/convert_ped_to_bed.sh
        ./gp2_genotools_data/batch_files/convert_ped_to_bed.sh {code}
        """

    # Create the job with the combined script
    create_script_job_with_bucket(
        project_id="gp2-release-terra",
        region='europe-west4',
        job_name=job_name,
        bucket_name='gp2_genotools_data',
        script_text=script
    )


In [34]:
# Initialize the client
client = batch_v1.BatchServiceClient()
job_id = f'pedtobed{study.lower()}{count}'

# Define the job name (replace with your project ID, location, and job ID)
project_id = "gp2-release-terra"
region = 'europe-west4'
job_name = f"projects/{project_id}/locations/{region}/jobs/{job_id}"

# Get the job details
job = client.get_job(name=job_name)

# Check the status
job_status = job.status.state
print(study)
print('last job id', count)
print(f"Job Status: {batch_v1.JobStatus.State(job_status).name}")

VETPD
last job id 14
Job Status: SUCCEEDED


In [35]:
missing_beds = []

for cohort in study_list:
    df = key[key['study'] == cohort]
    missing_cnt = 0

    with open(f"{raw_plink_path}/merge_bed_{cohort}.list", 'w') as f:
        for filename in df.IID:

            bed = f'{raw_plink_path}/{filename}'
            if os.path.isfile(f'{bed}.bed'):
                f.write(f'{bed}\n')
            else:
                #print(f'{bed} does not exist!!!')
                missing_cnt += 1
                missing_beds.append(filename)
    f.close()
    
    print(f"There is {missing_cnt} missing bed files for {cohort}.")

    #create missing_beds file for reference
    with open(f'{missing_idat_dir}/missing_beds_{cohort}.txt', 'w') as f:
        for m_bed in missing_beds:
            f.write(f'{m_bed}\n')
    f.close()

print(missing_beds)

There is 0 missing bed files for VETPD.
[]


# Merge files

In [36]:
key

,GP2sampleID_old,GP2sampleID,SentrixBarcode_A,SentrixPosition_A,study_old,study,sample_id,clinical_id,study_arm,study_type,...,family_history_other,family_history_pd,family_index,family_index_relationship,filename,FID,IID,pheno,sex_for_qc,GDPR
91592,NaN,VETPD_000193_s1,209281700115,R01C01,NaN,VETPD,GP2Z0061.1,GP2Z0061,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R01C01,2,1,0
91593,NaN,VETPD_000194_s1,209281700115,R02C01,NaN,VETPD,GP2Z0062.1,GP2Z0062,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R02C01,2,1,0
91594,NaN,VETPD_000195_s1,209281700115,R03C01,NaN,VETPD,GP2Z0063.1,GP2Z0063,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R03C01,2,1,0
91595,NaN,VETPD_000196_s1,209281700115,R04C01,NaN,VETPD,GP2Z0064.1,GP2Z0064,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700115_R04C01,2,1,0
91596,NaN,VETPD_000197_s1,209281700007,R06C01,NaN,VETPD,GP2Z0068.1,GP2Z0068,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281700007_R06C01,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91779,NaN,VETPD_000380_s1,209281710051,R08C01,NaN,VETPD,GP2Z0731.1,GP2Z0731,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R08C01,2,1,0
91780,NaN,VETPD_000381_s1,209281710051,R07C01,NaN,VETPD,GP2Z0732.1,GP2Z0732,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R07C01,2,1,0
91781,NaN,VETPD_000382_s1,209281710051,R06C01,NaN,VETPD,GP2Z0733.1,GP2Z0733,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R06C01,2,1,0
91782,NaN,VETPD_000383_s1,209281710051,R05C01,NaN,VETPD,GP2Z0734.1,GP2Z0734,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209281710051_R05C01,2,1,0


In [37]:
clin_key_dir = './gp2_genotools_data/clinical'
for cohort in study_list:
# write to files
    df = key[key['study'] == cohort]
    df[['FID','IID', 'FID', 'GP2sampleID']].to_csv(f'{clin_key_dir}/update_ids_{cohort}.txt', sep='\t', header=False, index=False)
    df[['FID', 'GP2sampleID', 'pheno']].to_csv(f'{clin_key_dir}/update_pheno_{cohort}.txt', sep='\t', header=False, index=False)
    df[['FID', 'GP2sampleID', 'sex_for_qc']].to_csv(f'{clin_key_dir}/update_sex_{cohort}.txt', sep='\t', header=False, index=False)

In [38]:
raw_plink_path

'./gp2_genotools_data/ped_bed'

In [39]:
missing_beds = []
for cohort in study_list:
    df = key[key['study'] == cohort]
    with open(f"{raw_plink_path}/merge_bed_{cohort}.list", 'w') as f:
        for filename in df.IID:
            bed = f'{raw_plink_path}/{filename}'
            if os.path.isfile(f'{bed}.bed'):
                f.write(f'{bed}\n')
            else:
                print(f'{bed} does not exist!!!')
                missing_beds.append(filename)
    f.close()

In [40]:
! head {raw_plink_path}/merge_bed_{cohort}.list

./gp2_genotools_data/ped_bed/209281700115_R01C01
./gp2_genotools_data/ped_bed/209281700115_R02C01
./gp2_genotools_data/ped_bed/209281700115_R03C01
./gp2_genotools_data/ped_bed/209281700115_R04C01
./gp2_genotools_data/ped_bed/209281700007_R06C01
./gp2_genotools_data/ped_bed/209281700115_R05C01
./gp2_genotools_data/ped_bed/209281700007_R07C01
./gp2_genotools_data/ped_bed/209281700007_R08C01
./gp2_genotools_data/ped_bed/209281700040_R01C01
./gp2_genotools_data/ped_bed/209281700115_R06C01


In [41]:
study

'VETPD'

In [42]:
! wc -l {raw_plink_path}/merge_bed_{cohort}.list

192 ./gp2_genotools_data/ped_bed/merge_bed_VETPD.list


In [43]:
# Create shell script to execute -- merge cohort files
count = 0 # for Job ID count

# Establish paths for inputs/outputs
#out_path = f'./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_{study}'
out_path = f'./gp2_genotools_data/tests/GP2_merge_{study}_extra'

plink_merge_cmd = f"/home/levineks/bin/plink1.9/plink --merge-list {raw_plink_path}/merge_bed_{cohort}.list --update-ids {clin_key_dir}/update_ids_{cohort}.txt --make-bed --out {out_path}"

with open(f'./gp2_genotools_data/batch_files/merge_by_cohort.sh', 'w') as f:
    f.write('#!/bin/bash\n\n')
    f.write(f'{plink_merge_cmd}\n')
    f.close()

print(f"Preparing bash script to merge bed files for {study}.")

Preparing bash script to merge bed files for VETPD.


In [44]:
# Double check the shell script to make sure it does what you need and paths are correct
! cat ./gp2_genotools_data/batch_files/merge_by_cohort.sh

#!/bin/bash

/home/levineks/bin/plink1.9/plink --merge-list ./gp2_genotools_data/ped_bed/merge_bed_VETPD.list --update-ids ./gp2_genotools_data/clinical/update_ids_VETPD.txt --make-bed --out ./gp2_genotools_data/tests/GP2_merge_VETPD_extra


In [45]:
study_list

['VETPD']

In [46]:
#Update the count with the last job # that you ran:
count = 5

# Set up the barcodes to process above

# Loop through chromosomes and create separate jobs
for study in study_list:
    # Generate a unique job name
    count += 1
    job_name = f'mergebycohort{study.lower()}{count}'
    
    # Create a script for the specific chromosome
    script = f"""
        #!/bin/bash

        # Install and activate PLINK2
        chmod +x ./gp2_genotools_data/batch_files/plink_module.sh
        ./gp2_genotools_data/batch_files/plink_module.sh
        
        # Make analysis script executable and run for a specific chromosome
        chmod +x ./gp2_genotools_data/batch_files/merge_by_cohort.sh
        ./gp2_genotools_data/batch_files/merge_by_cohort.sh
    """

    # Create the job
    create_script_job_with_bucket(
        project_id="gp2-release-terra",
        region='europe-west4',
        job_name=job_name,
        bucket_name='gp2_genotools_data',
        script_text=script
    )

In [49]:
# Initialize the client
client = batch_v1.BatchServiceClient()
job_id = f'mergebycohort{study.lower()}{count}'

# Define the job name (replace with your project ID, location, and job ID)
project_id = "gp2-release-terra"
region = 'europe-west4'
job_name = f"projects/{project_id}/locations/{region}/jobs/{job_id}"

# Get the job details
job = client.get_job(name=job_name)

# Check the status
job_status = job.status.state
print(study, 'cohort merge')
print(f"Job Status: {batch_v1.JobStatus.State(job_status).name}")

VETPD cohort merge
Job Status: SUCCEEDED


In [50]:
! ls -lh ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_{study}*
! ls -lh ./gp2_genotools_data/tests/GP2_merge_{study}_extra*

-rwxrwxrwx 1 levineks levineks  92M Feb 26 20:03 ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.bed
-rwxrwxrwx 1 levineks levineks  59M Feb 26 20:03 ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.bim
-rwxrwxrwx 1 levineks levineks 5.1K Feb 26 20:03 ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.fam
-rwxrwxrwx 1 levineks levineks 418K Feb 26 20:03 ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.hh
-rwxrwxrwx 1 levineks levineks 9.0M Feb 26 20:03 ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.log
-rwxrwxrwx 1 levineks levineks  110 Feb 26 20:03 ./gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.nosex


-rwxrwxrwx 1 levineks levineks  92M Mar  7 14:59 ./gp2_genotools_data/tests/GP2_merge_VETPD_extra.bed
-rwxrwxrwx 1 levineks levineks  59M Mar  7 14:59 ./gp2_genotools_data/tests/GP2_merge_VETPD_extra.bim
-rwxrwxrwx 1 levineks levineks 5.1K Mar  7 14:59 ./gp2_genotools_data/tests/GP2_merge_VETPD_extra.fam
-rwxrwxrwx 1 levineks levineks 620K Mar  7 14:59 ./gp2_genotools_data/tests/GP2_merge_VETPD_extra.hh
-rwxrwxrwx 1 levineks levineks 9.0M Mar  7 14:59 ./gp2_genotools_data/tests/GP2_merge_VETPD_extra.log
-rwxrwxrwx 1 levineks levineks   44 Mar  7 14:59 ./gp2_genotools_data/tests/GP2_merge_VETPD_extra.nosex


# Merge old and new files together

In [51]:
# get key for entire study
key = pd.read_csv(f'/home/levineks/gp2_genotools_data/clinical/GP2_master_key_MARCH_2025.txt', sep = '\t', low_memory = False)
key = key[key['study'] == study]
print(study)
print(len(key))
key.head()

VETPD
382


,GP2sampleID_old,GP2sampleID,SentrixBarcode_A,SentrixPosition_A,study_old,study,sample_id,clinical_id,study_arm,study_type,...,family_history_other,family_history_pd,family_index,family_index_relationship,filename,FID,IID,pheno,sex_for_qc,GDPR
91402,NaN,VETPD_000001_s1,209042130073,R02C01,NaN,VETPD,GP2Z0001.1,GP2Z0001,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130073_R02C01,2,1,0
91403,NaN,VETPD_000002_s1,209042130008,R06C01,NaN,VETPD,GP2Z0002.1,GP2Z0002,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130008_R06C01,2,1,0
91404,NaN,VETPD_000003_s1,209042130073,R07C01,NaN,VETPD,GP2Z0003.1,GP2Z0003,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130073_R07C01,2,1,0
91405,NaN,VETPD_000004_s1,209042130062,R06C01,NaN,VETPD,GP2Z0004.1,GP2Z0004,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130062_R06C01,2,1,0
91406,NaN,VETPD_000005_s1,209042130055,R04C01,NaN,VETPD,GP2Z0005.2,GP2Z0005,Case,Case(/Control),...,NaN,NaN,NaN,NaN,NaN,0,209042130055_R04C01,2,1,0


In [52]:
# Move the old files to test folder/we'll overwrite in r10 folder with the new ones
! cp /home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_{study}.* /home/levineks/gp2_genotools_data/tests/

In [53]:
# New update IDs
clin_key_dir = '/home/levineks/gp2_genotools_data/clinical'
key[['FID','IID', 'FID', 'GP2sampleID']].to_csv(f'{clin_key_dir}/update_ids_{study}.txt', sep='\t', header=False, index=False)
key[['FID', 'GP2sampleID', 'pheno']].to_csv(f'{clin_key_dir}/update_pheno_{study}.txt', sep='\t', header=False, index=False)
key[['FID', 'GP2sampleID', 'sex_for_qc']].to_csv(f'{clin_key_dir}/update_sex_{study}.txt', sep='\t', header=False, index=False)

In [54]:
raw_plink_path = '/home/levineks/gp2_genotools_data/merged'

missing_beds = []
with open(f"{raw_plink_path}/merge_bed_{study}.list", 'w') as f:

    bed1 = f'/home/levineks/gp2_genotools_data/tests/GP2_merge_{study}_extra'
    if os.path.isfile(f'{bed1}.bed'):
        f.write(f'{bed1}\n')
    else:
        print(f'{bed1} does not exist!!!')

    bed2 = f'/home/levineks/gp2_genotools_data/tests/GP2_merge_{study}'
    if os.path.isfile(f'{bed2}.bed'):
        f.write(f'{bed2}\n')
    else:
        print(f'{bed2} does not exist!!!')    
   
f.close()

In [55]:
! cat {raw_plink_path}/merge_bed_{study}.list 

/home/levineks/gp2_genotools_data/tests/GP2_merge_VETPD_extra
/home/levineks/gp2_genotools_data/tests/GP2_merge_VETPD


In [56]:
! cat {clin_key_dir}/update_ids_{study}.txt | wc -l

382


In [57]:
! cat /home/levineks/gp2_genotools_data/tests/GP2_merge_{study}_extra.fam | wc -l
! cat /home/levineks/gp2_genotools_data/tests/GP2_merge_{study}.fam | wc -l

192


190


In [58]:
#Merge files -- this should not take long because we're only merging 2 files
merge_path = '/home/levineks/gp2_genotools_data/merged_by_cohort_r10'

! /home/levineks/plink \
--merge-list {raw_plink_path}/merge_bed_{study}.list \
--update-ids {clin_key_dir}/update_ids_{study}.txt \
--make-bed \
--out {merge_path}/GP2_merge_{study}

PLINK v1.90b7 64-bit (16 Jan 2023)             www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD.log.
Options in effect:
  --make-bed
  --merge-list /home/levineks/gp2_genotools_data/merged/merge_bed_VETPD.list
  --out /home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD
  --update-ids /home/levineks/gp2_genotools_data/clinical/update_ids_VETPD.txt

32169 MB RAM detected; reserving 16084 MB for main workspace.
to length-80+ variant IDs; consider using a different naming scheme for long
indels and the like.
same position.
108706 more same-position warnings: see log file.
Performing single-pass merge (382 people, 2004347 variants).
Merged fileset written to                     
/home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD-merge.bed
+
/home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_VETPD-

In [59]:
print(study)
!cat /home/levineks/gp2_genotools_data/merged_by_cohort_r10/GP2_merge_{study}.fam | wc -l

VETPD
382
